In [1]:
import pandas as pd
import numpy as np

### Get senatorial candidates from wmp using the latest file

In [2]:
s = pd.read_csv("../datasets/candidates/wmpcand_012523_wmpid.csv")

In [3]:
s.columns

Index(['wmpid', 'genelect_cd', 'CurrCand', 'cand_name', 'cand_id',
       'cand_office', 'cand_office_st', 'cand_office_dist',
       'cand_party_affiliation', 'cand_incumbent_challenger_open_s',
       'dateadded_cd', 'office_wapo', 'result_wapo', 'primarydate_wapo',
       'trumpe_wapo', 'gender_wmp', 'gender_crp', 'latino_wmp', 'latino_crp',
       'race_wmp', 'race_crp1', 'race_crp2', 'race_crpmena', 'hse_cmpt_gen',
       'full_name', 'first_name', 'last_name'],
      dtype='object')

In [4]:
s.genelect_cd.value_counts(dropna=False)

0    2929
1    1366
Name: genelect_cd, dtype: int64

In [5]:
s = s.loc[(s['genelect_cd']==1)&(s['cand_office']=='S')]

In [6]:
s.shape

(173, 27)

In [7]:
s.head()

,wmpid,genelect_cd,CurrCand,cand_name,cand_id,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,latino_wmp,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena,hse_cmpt_gen,full_name,first_name,last_name
3613,WMPID1299,1,Y,"BOOZMAN, SEN. JOHN",S0AR00150,S,AR,0.0,REP,INCUMBENT,...,NaN,NaN,White,W,NaN,NaN,0,John Boozman,John,Boozman
3615,WMPID1302,1,Y,"KELLY, MARK",S0AZ00350,S,AZ,0.0,DEM,INCUMBENT,...,0.0,N,White,W,NaN,N,0,Mark Kelly,Kelly,Mark
3618,WMPID1305,1,Y,"BENNET, MICHAEL F.",S0CO00211,S,CO,0.0,DEM,INCUMBENT,...,0.0,N,White,W,NaN,N,0,Michael Bennet,Michael,Bennet
3619,WMPID1315,1,Y,"BLUMENTHAL, RICHARD",S0CT00177,S,CT,0.0,DEM,INCUMBENT,...,NaN,NaN,White,W,NaN,NaN,0,Richard Blumenthal,Richard,Blumenthal
3620,WMPID1321,1,Y,"RUBIO, MARCO",S0FL00338,S,FL,0.0,REP,INCUMBENT,...,1.0,Y,Hispanic/Latino,H,NaN,NaN,0,Marco Rubio,Marco,Rubio


In [8]:
s = s[['wmpid', 'cand_name', 'full_name', 'cand_office_st', 'cand_party_affiliation']]

In [9]:
s.head()

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation
3613,WMPID1299,"BOOZMAN, SEN. JOHN",John Boozman,AR,REP
3615,WMPID1302,"KELLY, MARK",Mark Kelly,AZ,DEM
3618,WMPID1305,"BENNET, MICHAEL F.",Michael Bennet,CO,DEM
3619,WMPID1315,"BLUMENTHAL, RICHARD",Richard Blumenthal,CT,DEM
3620,WMPID1321,"RUBIO, MARCO",Marco Rubio,FL,REP


In [10]:
len(s['wmpid'].unique())

173

### Load the data cleaned by by RA Jasmine

Note: 
- Jasmine's clean-up is based on a previous scraping result which is slightly different from ./data/bp2022_sen_scraped_face.csv
- wmpid was added to this file using both fuzzymatch and human coding

In [11]:
sen_c = pd.read_excel("./data/bp2022_sen_scraped_face_jasmine.xlsx")

In [12]:
sen_c.shape

(172, 20)

In [13]:
sen_c.head(2)

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge,hc_face_note,hc_face_url,hc_first_name,hc_last_name,hc_middle_name,hc_suffix,hc_affix,hc_name_other,hc_double_surname
0,WMPID1299,"BOOZMAN, SEN. JOHN",John Boozman,AR,REP,John Boozman,John Boozman (R),https://s3.amazonaws.com/ballotpedia-api4/file...,John Boozman,johnboozmanupdated.jpg,both,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WMPID1302,"KELLY, MARK",Mark Kelly,AZ,DEM,Mark Kelly,Mark Kelly (D),https://s3.amazonaws.com/ballotpedia-api4/file...,Mark Kelly,Mark-Kelly.png,both,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Use Jasmine's urls when scraped bp_url is missing

In [14]:
sen_c['bp_face_url']= sen_c['bp_url']

In [15]:
sen_c['bp_face_url']=sen_c.apply(
    lambda row: row['hc_face_url'] if pd.isnull(row['bp_face_url']) else row['bp_face_url'],
    axis=1)

In [16]:
sen_c.head(2)

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,...,hc_face_note,hc_face_url,hc_first_name,hc_last_name,hc_middle_name,hc_suffix,hc_affix,hc_name_other,hc_double_surname,bp_face_url
0,WMPID1299,"BOOZMAN, SEN. JOHN",John Boozman,AR,REP,John Boozman,John Boozman (R),https://s3.amazonaws.com/ballotpedia-api4/file...,John Boozman,johnboozmanupdated.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3.amazonaws.com/ballotpedia-api4/file...
1,WMPID1302,"KELLY, MARK",Mark Kelly,AZ,DEM,Mark Kelly,Mark Kelly (D),https://s3.amazonaws.com/ballotpedia-api4/file...,Mark Kelly,Mark-Kelly.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3.amazonaws.com/ballotpedia-api4/file...


In [17]:
sen_c.loc[sen_c['bp_face_url'].isna()]

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,...,hc_face_note,hc_face_url,hc_first_name,hc_last_name,hc_middle_name,hc_suffix,hc_affix,hc_name_other,hc_double_surname,bp_face_url
46,WMPID1644,"HERNANDEZ, SALOMON SR.",Salomon Sr. Hernandez,FL,W,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Salomon,Hernandez,NaN,Sr.,NaN,NaN,NaN,NaN
49,WMPID3554,"JOHNSON, ERIC CHASE",Eric Chase Johnson,GA,IND,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,WMPID3751,"HAUSKINS, DANIEL OBAMA MR JR",Daniel Hauskins,IL,GWP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,WMPID2419,"HORGES, BRADLEY LAMAR MR. JR.",Bradley Lamar Jr. Horges,IL,W,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Bradley,Horge,Lamar,Jr.,NaN,NaN,NaN,NaN
71,WMPID2339,"PARHAM, NICHOLAS",Nicholas Parham,LA,IND,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,WMPID4532,"VICTORIAN, MICHAEL DUKES",Michael Dukes Victorian,LA,SLP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,WMPID4178,"TUNNELL, LEONARD",Leonard Tunnell,MO,REP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,WMPID4165,"GRIFFIN, PETER JOHN",Peter John Griffin,NV,COM,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,WMPID2154,"LOWE, GRETCHEN RAE",Gretchen Rae Lowe,NV,NPA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,WMPID3203,"WISE MURRAY IV, WILLIAM",William Wise Murray Iv,SC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,William,Murray,Wise,IV,NaN,NaN,NaN,NaN


In [18]:
url=sen_c[['wmpid', 'bp_face_url']]

### Finalize face urls

In [19]:
df = s.merge(url, on='wmpid', how='left', indicator=True)

In [20]:
df._merge.value_counts()

both          172
left_only       1
right_only      0
Name: _merge, dtype: int64

check the missing one from Jasmine's file 
- No face image: https://ballotpedia.org/Walter_Sluzynsky

In [21]:
df.loc[df._merge=='left_only']

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,bp_face_url,_merge
125,WMPID2092,"SLUZYNSKY, WALTER STEVE MR.",Walter Steve Sluzynsky,PA,DEM,NaN,left_only


In [22]:
df.head()

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,bp_face_url,_merge
0,WMPID1299,"BOOZMAN, SEN. JOHN",John Boozman,AR,REP,https://s3.amazonaws.com/ballotpedia-api4/file...,both
1,WMPID1302,"KELLY, MARK",Mark Kelly,AZ,DEM,https://s3.amazonaws.com/ballotpedia-api4/file...,both
2,WMPID1305,"BENNET, MICHAEL F.",Michael Bennet,CO,DEM,https://s3.amazonaws.com/ballotpedia-api4/file...,both
3,WMPID1315,"BLUMENTHAL, RICHARD",Richard Blumenthal,CT,DEM,https://s3.amazonaws.com/ballotpedia-api4/file...,both
4,WMPID1321,"RUBIO, MARCO",Marco Rubio,FL,REP,https://s3.amazonaws.com/ballotpedia-api4/file...,both


In [23]:
df.loc[df.bp_face_url.isna()]

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,bp_face_url,_merge
46,WMPID1644,"HERNANDEZ, SALOMON SR.",Salomon Sr. Hernandez,FL,W,NaN,both
49,WMPID3554,"JOHNSON, ERIC CHASE",Eric Chase Johnson,GA,IND,NaN,both
59,WMPID3751,"HAUSKINS, DANIEL OBAMA MR JR",Daniel Hauskins,IL,GWP,NaN,both
62,WMPID2419,"HORGES, BRADLEY LAMAR MR. JR.",Bradley Lamar Jr. Horges,IL,W,NaN,both
71,WMPID2339,"PARHAM, NICHOLAS",Nicholas Parham,LA,IND,NaN,both
73,WMPID4532,"VICTORIAN, MICHAEL DUKES",Michael Dukes Victorian,LA,SLP,NaN,both
81,WMPID4178,"TUNNELL, LEONARD",Leonard Tunnell,MO,REP,NaN,both
95,WMPID4165,"GRIFFIN, PETER JOHN",Peter John Griffin,NV,COM,NaN,both
96,WMPID2154,"LOWE, GRETCHEN RAE",Gretchen Rae Lowe,NV,NPA,NaN,both
125,WMPID2092,"SLUZYNSKY, WALTER STEVE MR.",Walter Steve Sluzynsky,PA,DEM,NaN,left_only


In [24]:
df['face_2022']='yes'

In [25]:
df['face_category']='senate_general'

In [26]:
df = df[['wmpid', 'bp_face_url', 'face_2022', 'face_category']]

In [27]:
df.shape

(173, 4)

In [28]:
df.head()

,wmpid,bp_face_url,face_2022,face_category
0,WMPID1299,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,senate_general
1,WMPID1302,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,senate_general
2,WMPID1305,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,senate_general
3,WMPID1315,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,senate_general
4,WMPID1321,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,senate_general


In [29]:
df=df.rename(columns={'bp_face_url':'face_url'})

In [30]:
df.to_csv("./data/face_url_2022_senate_general.csv", index=True)